#Capitulo 7 - Punto 1
## Autores: Juan Barrera - Eduardo Cuadros
1. Desarrollar un agente de IA, con cualquier chatbot de IA, para el manejo de las tareas del curso.

In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
!nohup ollama serve > ollama_serve.log 2>&1 &

In [3]:
import time
time.sleep(9)

In [4]:
!ollama pull llama3.1

In [5]:
!pip install -qU langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


In [6]:
!ollama pull nomic-embed-text

In [8]:
!pip install -qU langchain_ollama

In [9]:
!pip install -qU langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.4 MB/s e

In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.llms import Ollama
from langchain_community.llms import Ollama
import os
# Limpiar colección existente
from langchain_chroma import Chroma
# Embeddings con Ollama
from langchain_ollama import OllamaEmbeddings

In [11]:


test_message = "¿Que son los automatas celulares?"

llm_llama = Ollama(model="llama3.1")

llm_llama.invoke(test_message)

/tmp/ipython-input-11-1070950171.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm_llama = Ollama(model="llama3.1")


'Los automatas celulares son un modelo computacional que simula el crecimiento de una estructura bidimensional. Cada célula puede estar en cualquiera de dos estados: vivo o muerta. El estado de cada célula del siguiente período depende de las reglas proporcionadas por el modelo.\n\nFue inventado para estudiar cómo pueden evolucionar los sistemas biológicos, pero también ha tenido aplicaciones en otros campos como la física y la química.\n\nEl automata celular más conocido es el llamado "juego del vida" de Conway.'

In [12]:
def cargar_textos_desde_archivos(directorio):
    documentos = []
    for nombre_archivo in os.listdir(directorio):
        if nombre_archivo.endswith(".txt"):
            ruta_archivo = os.path.join(directorio, nombre_archivo)
            with open(ruta_archivo, "r", encoding="utf-8") as f:
                contenido = f.read()
                fragmentos = (" ".join(contenido.split())).split(". ")
                documentos.extend([
                    f"Fragmento {i+1}/{len(fragmentos)} de '{nombre_archivo}': '{frag}'"
                    for i, frag in enumerate(fragmentos)
                ])
    return documentos

In [14]:
# Cargar textos desde archivos locales
textos_gabo = cargar_textos_desde_archivos("./Textos/")  # Asegúrate de tener esta carpeta con .txt


nomic_ollama_embeddings = OllamaEmbeddings(model="nomic-embed-text")



def reset_collection(collection_name, persist_directory):
    Chroma(collection_name=collection_name, embedding_function=nomic_ollama_embeddings, persist_directory=persist_directory).delete_collection()

reset_collection("IA-minirobots", "chroma")

# Indexar los textos
Chroma.from_texts(
    texts=textos_gabo,
    collection_name="IA-minirobots",
    embedding=nomic_ollama_embeddings,
    persist_directory="chroma"
)


In [15]:
vector_store = Chroma(
    collection_name="gabo_rag",
    embedding_function=nomic_ollama_embeddings,
    persist_directory="chroma"
)

retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [16]:
# Inicializa el modelo local
llm_llama = Ollama(model="llama3.1")

# NUEVO: Prompt para estudio académico
system_prompt = (
    "Eres un asistente académico experto en la asignatura que estoy cursando."
    " Recibirás fragmentos de textos relacionados con esta materia."
    "\nTu tarea es responder de forma clara, precisa y concisa, usando solamente la información contenida en los textos proporcionados.\n"
    "Si no tienes suficiente información, responde: 'No tengo suficiente información para responder con certeza'.\n"
    "Evita inventar datos o especular. No salgas del tema.\n\n"
    "Contexto:\n{context}"
)

# Configura el prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

# Construye la cadena de preguntas y respuestas
question_answer_chain = create_stuff_documents_chain(llm_llama, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [17]:
pregunta = "¿Cuales son los temas que aborda el curso?"
respuesta = rag_chain.invoke({"input": pregunta})
print(f"ASISTENTE_ACADEMICO: '{respuesta['answer']}'")

ASISTENTE_ACADEMICO: 'Lo siento, pero no tengo suficiente información para responder con certeza sobre los temas específicos que aborda el curso en cuestión. ¿Podrías proporcionarme más detalles o fragmentos de texto relacionados con la materia? De esta manera podré ofrecerte una respuesta precisa y clara.'


In [18]:
pregunta = "¿Que es el machine learning y para que sirve?"
respuesta = rag_chain.invoke({"input": pregunta})
print(f"ASISTENTE_ACADEMICO: '{respuesta['answer']}'")

ASISTENTE_ACADEMICO: 'El machine learning (ML) es una rama de la inteligencia artificial que permite a los ordenadores aprender de los datos y mejorar sus procesos sin necesidad de ser programados explícitamente. Su función principal es realizar predicciones o tomar decisiones basadas en el análisis de patrones y tendencias en grandes conjuntos de datos.

Es decir, se utiliza para automatizar tareas que requieren la capacidad de aprender a partir del experiencia, como la reconocimiento de imágenes, la clasificación de texto, la previsión de resultados, entre otras.'


In [19]:
pregunta = "¿Que aplicaciones tienen los automatas celulares?"
respuesta = rag_chain.invoke({"input": pregunta})
print(f"ASISTENTE_ACADEMICO: '{respuesta['answer']}'")

ASISTENTE_ACADEMICO: 'Los autómatas celulares pueden tener varias aplicaciones en diferentes campos, incluyendo:

*   Simulación de crecimiento y patrón de pauta, como en la computación celular.
*   Reconocimiento de patrones y procesamiento de información a gran escala.
*   Modelado de sistemas complejos, como el crecimiento de tejidos o las redes sociales.'


In [20]:
pregunta = "¿Que es un algoritmo genetico simple?"
respuesta = rag_chain.invoke({"input": pregunta})
print(f"ASISTENTE_ACADEMICO: '{respuesta['answer']}'")

ASISTENTE_ACADEMICO: 'Un algoritmo genético simple es una variante básica de los algoritmos genéticos, que se utiliza para encontrar la solución óptima a un problema determinado. Funciona mediante la simulación del proceso de selección natural y mejora continua de las generaciones de individuos virtuales, llamados "genomas". A través de la reproducción, mutación y selección, los genomas más aptos para resolver el problema tienen mayores posibilidades de sobrevivir en cada generación.'


In [21]:
pregunta = "¿Cuales son los conceptos basicos de programacion genetica?"
respuesta = rag_chain.invoke({"input": pregunta})
print(f"ASISTENTE_ACADEMICO: '{respuesta['answer']}'")

ASISTENTE_ACADEMICO: '"La programación genética, también conocida como evolución computacional, se basa en conceptos biológicos como la selección natural, el cruzamiento y la mutación para resolver problemas complejos.

Los conceptos básicos de la programación genética son:

*   La representación del individuo (solución): los individuos están codificados mediante un conjunto de bits o valores.
*   El cromosoma: es una representación del individuo, donde se almacena la información necesaria para su evaluación y evolución.
*   La población: es el conjunto de individuos que participan en el proceso de selección natural.
*   La función objetiva: esta es la función que debe maximizar o minimizar para resolver el problema.

La programación genética sigue estos pasos:

1.  Inicialización de la población: se crea una lista aleatoria de individuos, cada uno con su propia solución al problema.
2.  Evaluación de la población: cada individuo es evaluado según la función objetiva para determinar su

In [22]:
pregunta = "¿Que es una funcion de perdida?"
respuesta = rag_chain.invoke({"input": pregunta})
print(f"ASISTENTE_ACADEMICO: '{respuesta['answer']}'")

ASISTENTE_ACADEMICO: 'Una función de pérdida es un concepto matemático utilizado en la optimización y aprendizaje automático, especialmente en el algoritmo del gradiente descendente. En general, se define como una medida cuantitativa que representa el error entre los valores predichos por un modelo y los valores reales o correctos de una variable dependiente.

En otras palabras, la función de pérdida mide cuánto se aleja el resultado obtenido por el modelo del resultado deseado. Su objetivo es minimizarla para mejorar la precisión del modelo en su ajuste a los datos.

No tengo suficiente información para responder con certeza sobre el contexto específico en que estás solicitando esta definición, pero esta es la definición general de una función de pérdida.'


In [23]:
pregunta = "¿Cuales son los ejercicios y tareas de los temas vistos?"
respuesta = rag_chain.invoke({"input": pregunta})
print(f"ASISTENTE_ACADEMICO: '{respuesta['answer']}'")

ASISTENTE_ACADEMICO: 'Me alegra ayudarte. Sin embargo, no tengo la información específica sobre las tareas y ejercicios de cada tema. Necesitaría tener acceso a los contenidos o materiales del curso para brindarte una respuesta precisa. ¿Podrías proporcionarme más detalles o el enlace al contenido del tema? De esta manera puedo ayudarte mejor.'


In [ ]:
# Stop any running Ollama process
!pkill ollama

In [ ]:
# Restart Ollama server in the background
!nohup ollama serve > ollama_serve.log 2>&1 &

Para este ejercicio se planteo un chatbot con la api de ollama y se alimento con los temas vistos en clase en formato txt, luego se le realizaron preguntas al respecto a los contenidos del curso.